# Fully Segmented Markets

In [409]:
import numpy as np
q = np.array([0.4,0.2,0.2,0.2])
p = np.array([0.6, 0.8, 0.8, 0.8])
α = np.array([-1.0, -1.1, -1.2, -1.3]) 
# slope of own-demand curve: dqdp = α, firms prefer larger α (or smaller in magnitude)

def D(p):
    return 1+α*p

def P(q):
    return 1+α*q

P(q)

array([0.6 , 0.78, 0.76, 0.74])

In [410]:
def AvgP(p,q):
    return np.mean(p*q/np.sum(q))
    
AvgP(p,q)

0.18000000000000002

In [411]:
def D(p):
    return 1-p

def demand_derivatives(p, α):
    N = p.shape[0]
    dqdp = np.zeros((N,N))
    np.fill_diagonal(dqdp, α)
    return dqdp

dqdp = demand_derivatives(p, α)
dqdp # no substitution

array([[-1. ,  0. ,  0. ,  0. ],
       [ 0. , -1.1,  0. ,  0. ],
       [ 0. ,  0. , -1.2,  0. ],
       [ 0. ,  0. ,  0. , -1.3]])

In [412]:
def ownershipMatrix(p2f):
    J = len(p2f)
    F = len(np.unique(p2f))
    Ω = np.zeros((J,J))
    for i in range(J):
        for j in range(J):
            if p2f[i] == p2f[j]:
                Ω[i,j] = 1
    return Ω 

p2f = np.array([0,1,2,3])
Ω = ownershipMatrix(p2f)
Ω 

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [413]:
def get_hhi(q,p2f):
    return np.sum(np.bincount(p2f, weights=q/np.sum(q))**2)

get_hhi(q, p2f)

0.28

In [414]:
def get_mc(p, q, dqdp):
    return p + np.dot(np.linalg.inv(Ω*dqdp.T),q)

c = get_mc(p, q, dqdp)
c

array([0.2       , 0.61818182, 0.63333333, 0.64615385])

In [415]:
def foc(p,c,α,Ω):
    q = D(p)
    dqdp = demand_derivatives(p, α)
    FOC = -p + c - np.dot(np.linalg.inv(Ω*dqdp.T),q)
    return np.round(FOC,4)
foc(p,c,α,Ω)

array([ 0., -0., -0., -0.])

# Mergers raise HHI but leave avg. prices untouched. 

In [416]:
p2f_new = np.array([0,0,1,2])
Ω_new = ownershipMatrix(p2f_new)
Ω_new

array([[1., 1., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [417]:
import scipy as sp
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(c,α,Ω_new))
p_new 
q_new = D(p_new) 

In [418]:
p_new, q_new

(array([0.59999689, 0.80000988, 0.80000764, 0.79999747]),
 array([0.40000311, 0.19999012, 0.19999236, 0.20000253]))

In [419]:
AvgP(p_new,q_new), get_hhi(q_new, p2f_new) # Average Prices same. HHI rises.

(0.18000004513990303, 0.4400002874784087)

# Cost Increases

- avg.prices always rise
- low cost firm face cost increase: HHI falls because extremal quantitiy falls
- high cost firm face cost increase: HHI rises because avg quantities diverge from extremal values

In [420]:
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(c+np.array([0.1,0,0,0]),α,Ω))
q_new = D(p_new) 
p_new, q_new # prices rise, quantities fall
AvgP(p_new,q_new), get_hhi(q_new, p2f) # Average Price rises. HHI rises.

(0.18618554288268585, 0.26869413362896755)

In [421]:
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(c+np.array([0,0.1,0.1,0.1]),α,Ω))
q_new = D(p_new) 
p_new, q_new # prices rise, quantities fall
AvgP(p_new,q_new), get_hhi(q_new, p2f) # Average Price rises. HHI rises.

(0.18319758552374887, 0.3194155482548008)

# Elasticity (price-sensitivity) falls
- |α| is increasing (i.e. it is away from 0, i.e more elastic. Consumers are increasingly price sensitive)
- Price-sensitivity rises for leading firm: avg prices fall as leading firm has to reduce prices,raise quantities --> avg prices fall, HHI rises
- Price-sensitivity falls for leading firm: avg prices rise as leading firm can raise prices,reduce quantities --> avg prices rise, HHI falls 
- Price-sensitivity rises for laggard firm: avg prices fall as laggard firms must reduce prices,raise quantities --> avg prices fall, HHI falls
- Price-sensitivity falls for laggard firm: avg prices rise as leading firm can raise prices,reduce quantities --> avg prices rise, HHI rises 


In [464]:
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(np.array([0.3,0.3,0.3,0.3]),
                                                              np.array([-1,-1.2,-1.4,-1.6]),Ω))
q_new = D(p_new) 
print(AvgP(p_new,q_new), get_hhi(q_new, p2f))
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(np.array([0.3,0.3,0.3,0.3]),
                                                              np.array([-1.1,-1.2,-1.4,-1.6]),Ω))
q_new = D(p_new) 
print(AvgP(p_new,q_new), get_hhi(q_new, p2f))
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(np.array([0.3,0.3,0.3,0.3]),
                                                              np.array([-0.9,-1.2,-1.4,-1.6]),Ω))
q_new = D(p_new) 
print(AvgP(p_new,q_new), get_hhi(q_new, p2f))

0.15123930568877716 0.2514731157668921
0.150394304619129 0.2509607479965958
0.15209326998505684 0.2522664289574993


In [465]:
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(np.array([0.3,0.3,0.3,0.3]),
                                                              np.array([-1,-1.2,-1.4,-1.6]),Ω))
q_new = D(p_new) 
print(AvgP(p_new,q_new), get_hhi(q_new, p2f))
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(np.array([0.3,0.3,0.3,0.3]),
                                                              np.array([-1.0,-1.2,-1.4,-1.7]),Ω))
q_new = D(p_new) 
print(AvgP(p_new,q_new), get_hhi(q_new, p2f))
p_new = sp.optimize.fsolve(foc, x0=np.array([0,0,0,0]), args=(np.array([0.3,0.3,0.3,0.3]),
                                                              np.array([-1.0,-1.2,-1.4,-1.5]),Ω))
q_new = D(p_new) 
print(AvgP(p_new,q_new), get_hhi(q_new, p2f))

0.15123930568877716 0.2514731157668921
0.15048754442105788 0.25178741902049545
0.15202721290815108 0.2511918819057559
